In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import sklearn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pandas as pd
import scienceplots

from utils import *

In [ ]:
import matplotlib as mpl
import scienceplots

mpl.rcParams['text.usetex'] = True
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use(['science', 'grid'])

In [ ]:
class Model():
    def __init__(self, loss_fn, clf, theta=None):
        self.loss_fn = loss_fn
        self.clf = clf
        if not theta is None:
            self.theta = theta
        else:
            self.theta = torch.ones(X.shape[1])
    
    def decision_function(self, X):
        return self.clf(X, self.theta).flatten()
    
    def score(self, X, y, sample_weight=None):
        if sample_weight is None:
            sample_weight = torch.ones(X.shape[0], self.theta.shape[0])
        y_hats = self.decision_function(X)
        return self.loss_fn(y, y_hats, sample_weight=sample_weight)
    
    def fit(self, X, y, sample_weight, max_iters=1_000, lr=0.0001, quiet=True, round_lr=False): # n x m
        theta = torch.clone(self.theta)
        theta.requires_grad_(True)
        lrs = np.exp(np.linspace(np.log(0.1), np.log(lr), max_iters))
        for i in (pbar := tqdm(range(max_iters), position=0, leave=True, disable=quiet)):
            theta = theta.detach()
            theta.requires_grad_(True)
            optimizer = torch.optim.Adam([theta], lr=lrs[i])
            optimizer.zero_grad()
            
            y_hats = self.clf(X, theta)
            loss = self.loss_fn(y, y_hats, sample_weight=sample_weight)
            
            pbar.set_description(f'Loss: {loss:.2f}')
            loss.backward()
            optimizer.step()
        if round_lr:
            theta = torch.round(theta, decimals=int(1 - np.log10(lr)))
        self.theta = theta.detach()
        return self

In [ ]:
# Load data & fix
X = torch.tensor([[1, 1, 1], [1, 1, 1], [1, -1, 1], [-1, 1, 1], [-1, -1, 1]], dtype=torch.float)
y = torch.tensor([1, 1, -1, -1, -1], dtype=torch.float)
theta_0_np = np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float)

In [ ]:
idx_pos = np.where(y == 1)[0]
idx_neg = np.where(y == -1)[0]

In [ ]:
# Doublecheck fit
svm = sklearn.svm.SVC(kernel='rbf')
svm.fit(X, y=y)
svm.score(X, y=y), svm.score(X[y==-1.0], y[y==-1.0]), svm.score(X[y==1.0], y[y==1.0])

# All At Once

In [ ]:
# Initialization 

p = 0.5
m = theta_0_np.shape[0]
n, d = X.shape
clf = new_clf
loss_fn = hinge_loss
epochs = 10
EXP_NAME = f'n={n}_m={m}_p={p}_e={epochs}_linclf_hingeloss'

models = [Model(loss_fn=loss_fn, clf=clf, theta=torch.tensor(theta_0_np[j], dtype=torch.float)) for j in range(m)]
# Collect scores
y_hats = []
for j in range(m):
    y_hats.append(models[j].decision_function(X))
y_hats = torch.stack(y_hats).T
M = torch.zeros_like(y_hats)

# Setup End-Of-Run Stats

y_hats_eor = []
alphas_eor = []
Ms_eor = []
models_eor = []

# Iterable

for e in range(epochs):
    y_hats_eor.append(y_hats)
    # Optimize alpha
    alpha = opt_alpha(y_hats, quiet=False, round_lr=True)
    alphas_eor.append(alpha)
    # Update memory
    M = cache_memory(alpha=alpha, mem=M, p=p)
    Ms_eor.append(M)
    # Train new models
    models = [Model(loss_fn=loss_fn, clf=linear_clf).fit(X, y=y, sample_weight=M.T[j], max_iters=2_000) for j in range(m)]
    models = [Model(loss_fn=loss_fn, clf=maxlin_clf, theta=models[j].theta).fit(X, y=y, sample_weight=M.T[j], max_iters=2_000) for j in range(m)]
    models_eor.append(models)
    # Collect new scores
    y_hats = []
    for j in range(m):
        y_hats.append(models[j].decision_function(X))
    y_hats = torch.stack(y_hats).T

In [ ]:
# First graph the differences, then the alphas for model 1, then the alphas for model 2

plt.rcParams.update({'font.size': 15})
fig, ax = plt.subplots(1, 1+m, figsize=(17,3))

NUM_LINES = 6
prevs = []
ax[0].set_title(r'Service Loss $\sum_{i=1}^n \sum_{j=1}^m A_{i,j}^t \ell(h_j^t, x_i, y_i)$')
for j in range(m):
    M = Ms_eor[j]
    y_hats = [models_eor[e][j].score(X, y, sample_weight=alphas_eor[e+1].T[j]) for e in range(epochs-1)]
    style = '--' if np.any([np.allclose(p_yhts, y_hats) for p_yhts in prevs]) else '-'
    ax[0].plot(range(epochs-1), y_hats, style, marker=NUM_LINES, label=f'Model j={j}')
    prevs.append(y_hats)
    NUM_LINES += 1
ax[0].set_xticks(range(0, epochs-1, 2))
ax[0].set_xlabel(r'Epochs (t)')
# ax[0].legend()

prev_plots = 1

# Next plot alphas
for j in range(m):
    NUM_LINES = 6
    ax[prev_plots+j].set_title(r'Usages $A_{i,' + repr(str(j))[1:-1] + r'}$')
    prevs = []
    for i in range(n):
        alphas_ij = [alphas_eor[e][i,j] for e in range(epochs)]
        style = '--' if np.any([np.allclose(p_alphas, alphas_ij) for p_alphas in prevs]) else '-'
        ax[prev_plots+j].plot(range(epochs), alphas_ij, style, marker=NUM_LINES, label=r'$x_i = $' + repr(f'{X[i,:-1].tolist()}')[1:-1])
        NUM_LINES += 1
        prevs.append(alphas_ij)
    max_val = int(np.ceil(np.max(prevs)))
    ax[prev_plots+j].set_xticks([0] + list(range(1, epochs, 2)))
    ax[prev_plots+j].set_yticks(np.linspace(0, max_val, 1 + max_val * 2))
    ax[prev_plots+j].set_xlabel(r'Epochs (t)')
ax[-1].legend(loc=7)

fig.tight_layout(pad=0.5)